In [1]:
import pandas as pd
import numpy as np

import re

# Data Cleaning

In [6]:
data = pd.read_csv('vehicles_messy.csv')

C:\Users\Emilio\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# primera vista de cómo está mi dataframe. De entrada tengo muchos valores en 0 o NaN...
data.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [8]:
# Puedo contar la cantidad de nulos que tengo en cada columna... sin embargo no me está mostrando las 83.
data.isnull().sum()

barrels08         0
barrelsA08        0
charge120         0
charge240         0
city08            0
              ...  
modifiedOn        0
startStop     31705
phevCity          0
phevHwy           0
phevComb          0
Length: 83, dtype: int64

In [9]:
# En pandas puedo buscar por filtros, de forma a que solo me ponga en pantalla las columnas que sí tienen valores nulos
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

cylinders       123
displ           120
drive          1189
eng_dscr      15403
trany            11
guzzler       35562
trans_dscr    22796
tCharger      32657
sCharger      37177
atvType       34771
fuelType2     36435
rangeA        36440
evMotor       37281
mfrCode       30818
c240Dscr      37806
c240bDscr     37807
startStop     31705
dtype: int64

In [10]:
# Hay algunas columnas que tienen demasiados valores nulos, prácticamente no hay qué analizar... entonces vamos a
# eliminarlas
data.shape

(37843, 83)

In [11]:
# Creo una variable con los nombres de las columnas que quiero quitar de mi data
drop_cols = list(null_cols[null_cols > 10000].index)
drop_cols

['eng_dscr',
 'guzzler',
 'trans_dscr',
 'tCharger',
 'sCharger',
 'atvType',
 'fuelType2',
 'rangeA',
 'evMotor',
 'mfrCode',
 'c240Dscr',
 'c240bDscr',
 'startStop']

In [12]:
# Uso el método .drop con el parámetro axis=1 para borrar columnas.
data = data.drop(drop_cols, axis=1)
data.shape

(37843, 70)

In [13]:
# Vamos a ver qué columnas quedaron. Chequen que puedo usar filtros con más de una condición usando operadores lógicos
# Vemos que displ y cylinders tienen casi el mismo número de valores nulos, puede ser que sea por la misma razón...
# vamos a observar mejor
null_cols[(null_cols > 0)&(null_cols < 10000)]

cylinders     123
displ         120
drive        1189
trany          11
dtype: int64

In [14]:
# Hice un dataframe solamente con los casos en qué displ es nulo
# Luego, visualicé con algunas de las columnas que sé qué són más importantes para entender cómo es un auto
# específico... Y qué interesante, parece ser que todos los autos Eléctricos tienen displ y cylinders en nulo. Tiene
# lógica, verdad?
null_displ = data[data['displ'].isnull() == True]
null_displ[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,NaN,NaN
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,NaN,NaN
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,NaN,NaN
...,...,...,...,...,...,...,...,...
30969,2017,Kia,Soul Electric,Automatic (A1),Front-Wheel Drive,Electricity,NaN,NaN
30972,2016,Tesla,Model S (60 kW-hr battery pack),Automatic (A1),Rear-Wheel Drive,Electricity,NaN,NaN
30973,2016,Tesla,Model S AWD - 60D,Automatic (A1),All-Wheel Drive,Electricity,NaN,NaN
30974,2016,Tesla,Model S AWD - P100D,Automatic (A1),All-Wheel Drive,Electricity,NaN,NaN


In [15]:
# Voy a reemplazar estos nulos con 0, ya que no quiero perder la información de los autos eléctricos.
data[['displ', 'cylinders']] = data[['displ', 'cylinders']].fillna(0)
data[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']][data['fuelType'] == 'Electricity']

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,0.0,0.0
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,0.0,0.0
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,0.0,0.0
...,...,...,...,...,...,...,...,...
30969,2017,Kia,Soul Electric,Automatic (A1),Front-Wheel Drive,Electricity,0.0,0.0
30972,2016,Tesla,Model S (60 kW-hr battery pack),Automatic (A1),Rear-Wheel Drive,Electricity,0.0,0.0
30973,2016,Tesla,Model S AWD - 60D,Automatic (A1),All-Wheel Drive,Electricity,0.0,0.0
30974,2016,Tesla,Model S AWD - P100D,Automatic (A1),All-Wheel Drive,Electricity,0.0,0.0


In [16]:
null_cols[(null_cols > 0)&(null_cols < 10000)]

cylinders     123
displ         120
drive        1189
trany          11
dtype: int64

In [17]:
# Vemos que drive trae los tipos de tracción de los autos... ¿Qué autos trarán null?
data['drive'].value_counts()

Front-Wheel Drive             13286
Rear-Wheel Drive              12931
4-Wheel or All-Wheel Drive     6648
All-Wheel Drive                2057
4-Wheel Drive                  1065
2-Wheel Drive                   507
Part-time 4-Wheel Drive         159
2-Wheel Drive, Front              1
Name: drive, dtype: int64

In [18]:
# Hice de nuevo un sub-dataframe para ver qué pasa con esa columna del drive
null_drive = data[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]\
    [data['drive'].isnull()==True]

In [19]:
# Veo que casi todos los autos sobre los cuales no tengo esta información son los autos viejitos...
# Vamos a asumir que todos esos autos tendrían solamente tracción en las ruedas delanteras
# Tendría yo que hacer un análisis para ver si esto es cierto, pero para fines del ejercicio...
null_drive['year'].value_counts()

1984    1180
1999       3
2001       2
2008       1
2002       1
2000       1
1998       1
Name: year, dtype: int64

In [20]:
data['drive'] = data['drive'].fillna('Front-Wheel Drive')
print(data['drive'].isnull().sum())
data['drive'].value_counts()

0


Front-Wheel Drive             14475
Rear-Wheel Drive              12931
4-Wheel or All-Wheel Drive     6648
All-Wheel Drive                2057
4-Wheel Drive                  1065
2-Wheel Drive                   507
Part-time 4-Wheel Drive         159
2-Wheel Drive, Front              1
Name: drive, dtype: int64

In [21]:
# Ahora vamos por la última que nos falta, que es de la transmisión... Solo tengo 11 nulos
null_trany = data[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]\
    [data['trany'].isnull()==True]
null_trany

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,Front-Wheel Drive,Electricity,0.0,0.0
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8144,2001,Ford,Th!nk,NaN,Front-Wheel Drive,Electricity,0.0,0.0
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,0.0,0.0
8147,2001,Nissan,Hyper-Mini,NaN,Front-Wheel Drive,Electricity,0.0,0.0
9212,2002,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
9213,2002,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,0.0,0.0
10329,2003,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
19097,1984,Ford,F150 Pickup 2WD,NaN,2-Wheel Drive,Regular,8.0,5.8


In [22]:
# Casi todos los que no tienen transmisión son autos eléctricos... podemos asumir que tienen transmisión automática?
data['trany'].value_counts()

Automatic 4-spd                     11042
Manual 5-spd                         8311
Automatic 3-spd                      3151
Automatic (S6)                       2638
Manual 6-spd                         2429
Automatic 5-spd                      2184
Manual 4-spd                         1483
Automatic 6-spd                      1432
Automatic (S8)                        960
Automatic (S5)                        824
Automatic (variable gear ratios)      681
Automatic 7-spd                       663
Automatic (S7)                        261
Auto(AM-S7)                           256
Automatic 8-spd                       243
Automatic (S4)                        233
Auto(AM7)                             160
Auto(AV-S6)                           145
Auto(AM6)                             110
Automatic (A1)                        109
Auto(AM-S6)                            92
Automatic 9-spd                        90
Manual 3-spd                           77
Manual 7-spd                      

In [23]:
# Hay muchos tipos de transmisión automática, con muchas velocidades... pero no nos preocupemos por esto ahora,
# todavía vamos a moverle más a esta columna de transmisión. Por ahora vamos a reemplazar por "Automatic"

data['trany'] = data['trany'].fillna('Automatic')
data['trany'].value_counts()

Automatic 4-spd                     11042
Manual 5-spd                         8311
Automatic 3-spd                      3151
Automatic (S6)                       2638
Manual 6-spd                         2429
Automatic 5-spd                      2184
Manual 4-spd                         1483
Automatic 6-spd                      1432
Automatic (S8)                        960
Automatic (S5)                        824
Automatic (variable gear ratios)      681
Automatic 7-spd                       663
Automatic (S7)                        261
Auto(AM-S7)                           256
Automatic 8-spd                       243
Automatic (S4)                        233
Auto(AM7)                             160
Auto(AV-S6)                           145
Auto(AM6)                             110
Automatic (A1)                        109
Auto(AM-S6)                            92
Automatic 9-spd                        90
Manual 3-spd                           77
Manual 7-spd                      

In [24]:
# También tenemos que lidiar con valores incorrectos en la data, es parte de cumplir con su consistencia...

test = data[(data['cylinders']==0) & (data['displ']!=0)]
test[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
21506,1986,Mazda,RX-7,Manual 5-spd,Rear-Wheel Drive,Regular,0.0,1.3


In [25]:
# Tengo 1 auto que usa gasolina y según no tiene cilindros, pero sí tiene displacement...
# Lo que haría sería investigar para saber cuántos cilindros tiene este modelo específico. Supongamos que ya lo hice
# y tiene 4 cilindros. Voy a usar el método .loc para reemplazar:

data.loc[(data['cylinders']==0) & (data['displ']!=0), 'cylinders'] = 4
data.loc[21506,['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

year                     1986
make                    Mazda
model                    RX-7
trany            Manual 5-spd
drive        Rear-Wheel Drive
fuelType              Regular
cylinders                   4
displ                     1.3
Name: 21506, dtype: object

In [26]:
# Siguiente, quiero ver si tengo columnas que no me dan información de valor sobre lo que quiero hacer... Un ejemplo
# de esto serían datos que tienen baja varianza... Es decir, casi todas las filas van a tener el mismo valor para esa
# columna. Vamos a encontrarlos

low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

In [27]:
# Tenemos todas estas columnas que realmente no me están agregando información... A no ser que fueran datos muy
# importantes justo porque solo algunos autos tendrán info diferente y es parte de lo que quiero modelar, lo podría
# eliminar.
low_variance

['barrelsA08',
 'charge120',
 'charge240',
 'cityA08',
 'cityA08U',
 'cityCD',
 'cityE',
 'cityUF',
 'co2A',
 'co2TailpipeAGpm',
 'combA08',
 'combA08U',
 'combE',
 'combinedCD',
 'combinedUF',
 'fuelCostA08',
 'ghgScoreA',
 'highwayA08',
 'highwayA08U',
 'highwayCD',
 'highwayE',
 'highwayUF',
 'phevBlended',
 'range',
 'rangeCity',
 'rangeCityA',
 'rangeHwy',
 'rangeHwyA',
 'UCityA',
 'UHighwayA',
 'charge240b',
 'phevCity',
 'phevHwy',
 'phevComb']

In [28]:
# SIempre acordarse de que si eliminamos columnas el axis=1, si eliminamos filas, axis=0
data = data.drop(low_variance, axis=1)


In [29]:
# Otro punto importante de analizar son los valores extremos en mis columnas, que llamamos outliers. Los outliers
# suelen tener gran impacto en cualquier modelo que realizamos, ya que los modelos tienden a interpretar la
# importancia de los datos de acuerdo a su magnitud

# Una forma común de definir outliers es a través de identificar datos que estén arriba o abajo del rango
# intercuartil (1.5x, pero puedes definir lo que sea más conveniente en tu data)
# aquí puedo ver cuáles son mis cuartiles para cada columna numérica
stats = data.describe().transpose()
stats

,count,mean,std,min,25%,50%,75%,max
barrels08,37843.0,17.532506,4.575950,0.06,14.33087,17.347895,20.600625,47.087143
city08,37843.0,17.941389,6.660360,6.00,15.00000,17.000000,20.000000,138.000000
city08U,37843.0,4.042737,9.645820,0.00,0.00000,0.000000,0.000000,138.304000
co2,37843.0,61.503713,153.387715,-1.00,-1.00000,-1.000000,-1.000000,847.000000
co2TailpipeGpm,37843.0,473.179736,122.188847,0.00,388.00000,467.736842,555.437500,1269.571429
comb08,37843.0,20.195809,6.623444,7.00,17.00000,19.000000,23.000000,124.000000
comb08U,37843.0,4.549751,10.389994,0.00,0.00000,0.000000,0.000000,124.360100
cylinders,37843.0,5.719446,1.778959,0.00,4.00000,6.000000,6.000000,16.000000
displ,37843.0,3.307658,1.371982,0.00,2.20000,3.000000,4.300000,8.400000
engId,37843.0,8860.308961,17829.683477,0.00,0.00000,211.000000,4505.000000,69102.000000


In [30]:
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
barrels08,37843.0,17.532506,4.575950,0.06,14.33087,17.347895,20.600625,47.087143,6.269755
city08,37843.0,17.941389,6.660360,6.00,15.00000,17.000000,20.000000,138.000000,5.000000
city08U,37843.0,4.042737,9.645820,0.00,0.00000,0.000000,0.000000,138.304000,0.000000
co2,37843.0,61.503713,153.387715,-1.00,-1.00000,-1.000000,-1.000000,847.000000,0.000000
co2TailpipeGpm,37843.0,473.179736,122.188847,0.00,388.00000,467.736842,555.437500,1269.571429,167.437500
comb08,37843.0,20.195809,6.623444,7.00,17.00000,19.000000,23.000000,124.000000,6.000000
comb08U,37843.0,4.549751,10.389994,0.00,0.00000,0.000000,0.000000,124.360100,0.000000
cylinders,37843.0,5.719446,1.778959,0.00,4.00000,6.000000,6.000000,16.000000,2.000000
displ,37843.0,3.307658,1.371982,0.00,2.20000,3.000000,4.300000,8.400000,2.100000
engId,37843.0,8860.308961,17829.683477,0.00,0.00000,211.000000,4505.000000,69102.000000,4505.000000


In [31]:
# Vamos ahora a hacer un subset de nuestra data con los outliers que encontremos.
# Estoy definiendo como outlier todo lo que esté arriba o abajo de 1.5x el rango intercuartil

outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) |
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

In [32]:
# Ahora que tengo qué columnas y qué valores de cada columna son outliers, puedo decidir qué hacer con ellos.
# Aquí tendría que ir columna por columna tomando una decisión...
# Algunas decisiones posibles:
# Borrar registros (si identifico que son pocos, puede ser por algún error de medición, por ejemplo...
# Reemplazarlos por el máximo o el mínimo
# Si son columnas escalares, como por ejemplo cylinders, yo podría transformarlas de modo a que tener más cilindros
# no represente más importancia que tener menos cilindros.
# Si son muchos, sí quiero considerarlos para mi distribución, al final ya no son tan "outliers" así... ya es un
# fenómeno. Entonces puedo utilizar técnicas de normalización de datos para que todos estén en un mismo rango y así
# pueda trabajar  con ellos en el modelo. Vamos a hacer las dos últimas como ejemplo
outliers['Outlier'].value_counts()

comb08U           7299
city08U           7299
highway08U        7299
engId             6677
lv2               6250
pv2               6239
co2               5854
feScore           5815
ghgScore          5815
hlv               4615
hpv               4614
city08            1470
UCity             1440
lv4               1357
highway08          996
comb08             969
UHighway           893
cylinders          852
co2TailpipeGpm     830
youSaveSpend       604
fuelCost08         604
barrels08          432
displ               43
Name: Outlier, dtype: int64

In [33]:
# Los cilindros son valores enteros y limitados, no tendré muchos de ellos...
# Vamos a suponer que quiero que tengan todos el mismo peso, no quiero que me considere un carro con muchos cilindros
# como más importante que los que tienen pocos. Así que todo lo que sea más de 8 cilindros voy a clasificar como "Más
#  de 8" y luego haré categorías.
data['cylinders'].value_counts()

4.0     14487
6.0     13184
8.0      8293
5.0       766
12.0      569
3.0       207
10.0      153
0.0       122
2.0        54
16.0        8
Name: cylinders, dtype: int64

In [34]:
# Clasifico los cilindros en 3 clases

def clasificar_cilindros(x):
    if x >= 8.0:
        return "A lot of cylinders"
    elif x == 0.0:
        return "No cylinders"
    else:
        return "Regular cylinders"

data['cylinders_class'] = data['cylinders'].apply(clasificar_cilindros)
data['cylinders_class'].value_counts()


Regular cylinders     28698
A lot of cylinders     9023
No cylinders            122
Name: cylinders_class, dtype: int64

In [35]:
# Ahora puedo tener 2 abordajes... o quiero dar la misma importancia a todos... Si es así, tendría que transformar
# esto en 3 columnas binarias, esto es llamado one-hot-encoding

cylinders_dummy = pd.get_dummies(data['cylinders_class'])
cylinders_dummy

,A lot of cylinders,No cylinders,Regular cylinders
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
...,...,...,...
37838,0,0,1
37839,0,0,1
37840,0,0,1
37841,0,0,1


In [36]:
# Si quisiera mantener este orden en mi variable, es decir, que los que no tienen cilindros valgan menos que los que
# sí... entonces puedo transformar la columna en categórica y cada categoría tendrá un número de forma ordinal

cylinders_label = data['cylinders_class'].astype('category')
cylinders_label.value_counts()

Regular cylinders     28698
A lot of cylinders     9023
No cylinders            122
Name: cylinders_class, dtype: int64

In [37]:
cylinders_label = cylinders_label.cat.codes
cylinders_label.value_counts()

2    28698
0     9023
1      122
dtype: int64

In [38]:
# Ahora vamos a suponer que quiero lidar con los outliers de esta columna que tiene muchos valores diferentes, son
# continuos y no quiero hacer con que sean categóricos... tampoco los quiero borrar... Entonces lo que debería hacer
# sería normalizar estos datos, ponerlos en 1 mismo rango para que los modelos no interpreten datos que son de miles
# de la misma forma que datos en unidades... todo tiene que estar correcto.
data['highway08U'].value_counts()

0.0000     30544
23.0111       33
28.0000       32
27.0000       30
27.0956       29
           ...  
28.5550        1
18.5006        1
16.8536        1
26.9664        1
20.5975        1
Name: highway08U, Length: 3090, dtype: int64

In [39]:
# Una de las formas de reescalar esta data es a través del método min max... los invito a investigar otros métodos y
# para qué se usan...

def min_max_scaling(df):
    df_norm = df.copy()
    for column in df_norm._get_numeric_data():
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / \
                          (df_norm[column].max() - df_norm[column].min())
    return df_norm

data_minmax = min_max_scaling(data)
data_minmax.head()

,barrels08,city08,city08U,co2,co2TailpipeGpm,comb08,comb08U,cylinders,displ,drive,...,pv4,trany,UCity,UHighway,VClass,year,youSaveSpend,createdOn,modifiedOn,cylinders_class
0,0.332483,0.098485,0.0,0.0,0.333333,0.119658,0.0,0.25,0.238095,Rear-Wheel Drive,...,0.00000,Manual 5-spd,0.118097,0.219987,Two Seaters,0.030303,0.800000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders
1,0.635900,0.022727,0.0,0.0,0.636364,0.034188,0.0,0.75,0.583333,Rear-Wheel Drive,...,0.00000,Manual 5-spd,0.055674,0.119422,Two Seaters,0.030303,0.523810,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,A lot of cylinders
2,0.258314,0.128788,0.0,0.0,0.259259,0.170940,0.0,0.25,0.261905,Front-Wheel Drive,...,0.00000,Manual 5-spd,0.146778,0.295412,Subcompact Cars,0.030303,0.866667,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders
3,0.635900,0.030303,0.0,0.0,0.636364,0.034188,0.0,0.50,0.619048,Rear-Wheel Drive,...,0.00000,Automatic 3-spd,0.061860,0.104756,Vans,0.030303,0.523810,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,A lot of cylinders
4,0.367615,0.083333,0.0,0.0,0.368421,0.102564,0.0,0.25,0.261905,4-Wheel or All-Wheel Drive,...,0.46875,Manual 5-spd,0.106288,0.201131,Compact Cars,0.272727,0.695238,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders


In [40]:
data.head()


,barrels08,city08,city08U,co2,co2TailpipeGpm,comb08,comb08U,cylinders,displ,drive,...,pv4,trany,UCity,UHighway,VClass,year,youSaveSpend,createdOn,modifiedOn,cylinders_class
0,15.695714,19,0.0,-1,423.190476,21,0.0,4.0,2.0,Rear-Wheel Drive,...,0,Manual 5-spd,23.3333,35.0000,Two Seaters,1985,-1250,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders
1,29.964545,9,0.0,-1,807.909091,11,0.0,12.0,4.9,Rear-Wheel Drive,...,0,Manual 5-spd,11.0000,19.0000,Two Seaters,1985,-8500,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,A lot of cylinders
2,12.207778,23,0.0,-1,329.148148,27,0.0,4.0,2.2,Front-Wheel Drive,...,0,Manual 5-spd,29.0000,47.0000,Subcompact Cars,1985,500,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders
3,29.964545,10,0.0,-1,807.909091,11,0.0,8.0,5.2,Rear-Wheel Drive,...,0,Automatic 3-spd,12.2222,16.6667,Vans,1985,-8500,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,A lot of cylinders
4,17.347895,17,0.0,-1,467.736842,19,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,...,90,Manual 5-spd,21.0000,32.0000,Compact Cars,1993,-4000,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,Regular cylinders


In [41]:
# Otro tema que tenemos que preocuparnos mientras preparamos nuestra data es la corrección de los tipos...

data.dtypes

barrels08          float64
city08               int64
city08U            float64
co2                  int64
co2TailpipeGpm     float64
comb08               int64
comb08U            float64
cylinders          float64
displ              float64
drive               object
engId                int64
feScore              int64
fuelCost08           int64
fuelType            object
fuelType1           object
ghgScore             int64
highway08            int64
highway08U         float64
hlv                  int64
hpv                  int64
id                   int64
lv2                  int64
lv4                  int64
make                object
model               object
mpgData             object
pv2                  int64
pv4                  int64
trany               object
UCity              float64
UHighway           float64
VClass              object
year                 int64
youSaveSpend         int64
createdOn           object
modifiedOn          object
cylinders_class     object
d

In [42]:
# Puedo por ejemplo cambiar esta columna de fecha por un formato de fecha real... suponiendo que las fechas son
# importantes para mi análisis de serie de tiempo... por supuesto aquí solo tengo el año, pero como ejemplo...
data['year'] = pd.to_datetime(data['year'], format='%Y')
data['year']

0       1985-01-01
1       1985-01-01
2       1985-01-01
3       1985-01-01
4       1993-01-01
           ...    
37838   1993-01-01
37839   1993-01-01
37840   1993-01-01
37841   1993-01-01
37842   1993-01-01
Name: year, Length: 37843, dtype: datetime64[ns]

In [43]:
# También puede suceder que yo tenga columnas que deberían ser numéricas pero aparecen como texto, esto es porque hay
# algún caracter especial que no sabemos cuál y hace con que toda la columna sea interpretada como objeto (string)...
# Deberíamos limpiar estos datos para luego poder cambiar el tipo de la columna.

# Les dije que regresaríamos a limpiar la columna de transmisión... Pues veamos qué traemos aquí

data['trany'].value_counts()

Automatic 4-spd                     11042
Manual 5-spd                         8311
Automatic 3-spd                      3151
Automatic (S6)                       2638
Manual 6-spd                         2429
Automatic 5-spd                      2184
Manual 4-spd                         1483
Automatic 6-spd                      1432
Automatic (S8)                        960
Automatic (S5)                        824
Automatic (variable gear ratios)      681
Automatic 7-spd                       663
Automatic (S7)                        261
Auto(AM-S7)                           256
Automatic 8-spd                       243
Automatic (S4)                        233
Auto(AM7)                             160
Auto(AV-S6)                           145
Auto(AM6)                             110
Automatic (A1)                        109
Auto(AM-S6)                            92
Automatic 9-spd                        90
Manual 3-spd                           77
Manual 7-spd                      

In [44]:
data['trany'] = data['trany'].str.replace('-', '')
data['trany'].value_counts()

Automatic 4spd                      11042
Manual 5spd                          8311
Automatic 3spd                       3151
Automatic (S6)                       2638
Manual 6spd                          2429
Automatic 5spd                       2184
Manual 4spd                          1483
Automatic 6spd                       1433
Automatic (S8)                        960
Automatic (S5)                        824
Automatic (variable gear ratios)      681
Automatic 7spd                        663
Automatic (S7)                        261
Auto(AMS7)                            256
Automatic 8spd                        243
Automatic (S4)                        233
Auto(AM7)                             160
Auto(AVS6)                            145
Auto(AM6)                             110
Automatic (A1)                        109
Auto(AMS6)                             92
Automatic 9spd                         90
Manual 3spd                            77
Manual 7spd                       

In [45]:
def limpiar_col(x):
    reemplazar = [('Automatic', 'Auto'), ('Auto(', 'Auto '),
                  ('Manual(', 'Manual '), ('(', ''), (')', '')]
    for tupla in reemplazar:
        x = x.replace(tupla[0], tupla[1])
    return x

data['trany'] = data['trany'].apply(limpiar_col)
data['trany'].value_counts()

Auto 4spd                    11042
Manual 5spd                   8311
Auto 3spd                     3151
Auto S6                       2638
Manual 6spd                   2429
Auto 5spd                     2184
Manual 4spd                   1483
Auto 6spd                     1433
Auto S8                        960
Auto S5                        824
Auto variable gear ratios      681
Auto 7spd                      663
Auto S7                        261
Auto AMS7                      256
Auto 8spd                      243
Auto S4                        233
Auto AM7                       160
Auto AVS6                      155
Auto AM6                       111
Auto A1                        110
Auto AMS6                       92
Auto 9spd                       90
Manual 3spd                     77
Manual 7spd                     68
Auto AVS7                       63
Auto AVS8                       27
Auto S9                         26
Manual 4spd Doubled             17
Auto AM5            

In [46]:
data['trany_type'] = data['trany'].apply(lambda x: re.findall(r'^\w+\b', x)[0])
data['trany_type'].value_counts()

Auto      25454
Manual    12389
Name: trany_type, dtype: int64

In [47]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


In [48]:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders',
                  'trany', 'drive', 'VClass','fuelType','barrels08',
                  'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

data = data[select_columns].drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  885


# Data Manipulation

In [50]:
data.columns

Index(['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive',
       'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'comb08',
       'co2TailpipeGpm', 'fuelCost08'],
      dtype='object')

In [51]:
data.columns = ['Manufacturer','Model','Year','Displacement',
                'Cylinders','Transmission','Drivetrain',
                'Vehicle_Class','Fuel_Type','Fuel_Barrels_Year',
                'City_MPG','Highway_MPG','Combined_MPG',
                'CO2_Emission_Grams_Mile','Fuel_Cost_Year']

data.columns

Index(['Manufacturer', 'Model', 'Year', 'Displacement', 'Cylinders',
       'Transmission', 'Drivetrain', 'Vehicle_Class', 'Fuel_Type',
       'Fuel_Barrels_Year', 'City_MPG', 'Highway_MPG', 'Combined_MPG',
       'CO2_Emission_Grams_Mile', 'Fuel_Cost_Year'],
      dtype='object')

In [52]:
data = data.rename(columns={'Manufacturer':'Make',
                            'Displacement':'Engine_Displacement'})

data.columns

Index(['Make', 'Model', 'Year', 'Engine_Displacement', 'Cylinders',
       'Transmission', 'Drivetrain', 'Vehicle_Class', 'Fuel_Type',
       'Fuel_Barrels_Year', 'City_MPG', 'Highway_MPG', 'Combined_MPG',
       'CO2_Emission_Grams_Mile', 'Fuel_Cost_Year'],
      dtype='object')

In [53]:
column_order = ['Year','Make','Model','Vehicle_Class',
                'Transmission','Drivetrain','Fuel_Type',
                'Cylinders','Engine_Displacement','Fuel_Barrels_Year',
                'City_MPG','Highway_MPG','Combined_MPG',
                'CO2_Emission_Grams_Mile','Fuel_Cost_Year']

data = data[column_order]
data.head()

,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,City_MPG,Highway_MPG,Combined_MPG,CO2_Emission_Grams_Mile,Fuel_Cost_Year
0,1985-01-01,Alfa Romeo,Spider Veloce 2000,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,4.0,2.0,15.695714,19,25,21,423.190476,1600
1,1985-01-01,Ferrari,Testarossa,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,12.0,4.9,29.964545,9,14,11,807.909091,3050
2,1985-01-01,Dodge,Charger,Subcompact Cars,Manual 5spd,Front-Wheel Drive,Regular,4.0,2.2,12.207778,23,33,27,329.148148,1250
3,1985-01-01,Dodge,B150/B250 Wagon 2WD,Vans,Auto 3spd,Rear-Wheel Drive,Regular,8.0,5.2,29.964545,10,12,11,807.909091,3050
4,1993-01-01,Subaru,Legacy AWD Turbo,Compact Cars,Manual 5spd,4-Wheel or All-Wheel Drive,Premium,4.0,2.2,17.347895,17,23,19,467.736842,2150


In [56]:
filtered = data[(data['Make']=='Ford') &
                (data['Cylinders']>=6) &
                (data['Combined_MPG'] < 18)]

filtered.head(10)

,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,City_MPG,Highway_MPG,Combined_MPG,CO2_Emission_Grams_Mile,Fuel_Cost_Year
61,1993-01-01,Ford,Thunderbird,Midsize Cars,Auto 4spd,Rear-Wheel Drive,Premium,6.0,3.8,19.388824,15,21,17,522.764706,2400
91,1985-01-01,Ford,E150 Club Wagon,Vans,Auto 3spd,Rear-Wheel Drive,Regular,6.0,4.9,25.354615,12,15,13,683.615385,2550
102,1985-01-01,Ford,E150 Club Wagon,Vans,Auto 4spd,Rear-Wheel Drive,Regular,6.0,4.9,23.543571,13,17,14,634.785714,2400
114,1985-01-01,Ford,E150 Club Wagon,Vans,Manual 4spd,Rear-Wheel Drive,Regular,6.0,4.9,20.600625,15,17,16,555.437500,2100
125,1985-01-01,Ford,E150 Club Wagon,Vans,Auto 4spd,Rear-Wheel Drive,Regular,8.0,5.0,27.467500,11,14,12,740.583333,2800
136,1985-01-01,Ford,E150 Club Wagon,Vans,Auto 3spd,Rear-Wheel Drive,Regular,8.0,5.8,29.964545,10,12,11,807.909091,3050
308,1993-01-01,Ford,F150 Pickup 2WD,Standard Pickup Trucks,Auto 4spd,Rear-Wheel Drive,Regular,6.0,4.9,21.974000,14,18,15,592.466667,2200
309,1993-01-01,Ford,F150 Pickup 2WD,Standard Pickup Trucks,Manual 4spd,Rear-Wheel Drive,Regular,6.0,4.9,21.974000,15,17,15,592.466667,2200
310,1993-01-01,Ford,F150 Pickup 2WD,Standard Pickup Trucks,Manual 5spd,Rear-Wheel Drive,Regular,6.0,4.9,21.974000,14,18,15,592.466667,2200
311,1993-01-01,Ford,F150 Pickup 2WD,Standard Pickup Trucks,Auto 4spd,Rear-Wheel Drive,Regular,8.0,5.0,21.974000,13,18,15,592.466667,2200


In [57]:
mpg_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
bins = pd.cut(data['Combined_MPG'],5, labels=mpg_labels)
bins.value_counts()

Very Low     35565
Low           1280
Very High       50
High            40
Moderate        23
Name: Combined_MPG, dtype: int64

In [58]:
bins = pd.qcut(data['Combined_MPG'],5, labels=mpg_labels)
bins.value_counts()

Moderate     9740
Very Low     9252
Low          6085
Very High    6079
High         5802
Name: Combined_MPG, dtype: int64

In [59]:
cutoffs = [7,14,21,23,30,40]
bins = pd.cut(data['Combined_MPG'], cutoffs, labels=mpg_labels)
bins.value_counts()

Low          20465
High          6316
Very Low      4607
Moderate      4172
Very High     1130
Name: Combined_MPG, dtype: int64

In [60]:
data.loc[data['Transmission'].str.startswith('A'), 'TransType'] = 'Automatic'
data.loc[data['Transmission'].str.startswith('M'), 'TransType'] = 'Manual'
data['TransType'].value_counts()

Automatic    24929
Manual       12029
Name: TransType, dtype: int64

In [61]:
avg_mpg = data.groupby('Make', as_index=False).agg({'Combined_MPG':'mean'})
avg_mpg.columns = ['Make', 'Avg_MPG']
avg_mpg

,Make,Avg_MPG
0,AM General,14.750000
1,ASC Incorporated,16.000000
2,Acura,21.506623
3,Alfa Romeo,19.558140
4,American Motors Corporation,17.681818
...,...,...
128,Volkswagen,24.351376
129,Volvo,20.615697
130,Wallace Environmental,13.875000
131,Yugo,25.000000


In [62]:
data = pd.merge(data, avg_mpg, on='Make')
data.head(10)


,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,City_MPG,Highway_MPG,Combined_MPG,CO2_Emission_Grams_Mile,Fuel_Cost_Year,TransType,Avg_MPG
0,1985-01-01,Alfa Romeo,Spider Veloce 2000,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,4.0,2.0,15.695714,19,25,21,423.190476,1600,Manual,19.55814
1,1994-01-01,Alfa Romeo,Spider,Two Seaters,Manual 5spd,Rear-Wheel Drive,Premium,4.0,2.0,14.982273,19,27,22,403.954545,1850,Manual,19.55814
2,1994-01-01,Alfa Romeo,164,Compact Cars,Auto 4spd,Front-Wheel Drive,Premium,6.0,3.0,20.600625,14,20,16,555.437500,2550,Automatic,19.55814
3,1994-01-01,Alfa Romeo,164,Compact Cars,Manual 5spd,Front-Wheel Drive,Premium,6.0,3.0,18.311667,15,22,18,493.722222,2250,Manual,19.55814
4,1995-01-01,Alfa Romeo,164,Compact Cars,Auto 4spd,Front-Wheel Drive,Premium,6.0,3.0,20.600625,14,20,16,555.437500,2550,Automatic,19.55814
5,1995-01-01,Alfa Romeo,164,Compact Cars,Manual 5spd,Front-Wheel Drive,Premium,6.0,3.0,18.311667,15,22,18,493.722222,2250,Manual,19.55814
6,1986-01-01,Alfa Romeo,Spider,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,4.0,2.0,15.695714,19,26,21,423.190476,1600,Manual,19.55814
7,1986-01-01,Alfa Romeo,GTV,Minicompact Cars,Manual 5spd,Rear-Wheel Drive,Regular,6.0,2.5,17.347895,17,24,19,467.736842,1750,Manual,19.55814
8,1984-01-01,Alfa Romeo,Spider Veloce 2000,Two Seaters,Manual 5spd,Front-Wheel Drive,Regular,4.0,2.0,15.695714,18,25,21,423.190476,1600,Manual,19.55814
9,1984-01-01,Alfa Romeo,GT V6 2.5,Minicompact Cars,Manual 5spd,Front-Wheel Drive,Regular,6.0,2.5,16.480500,17,24,20,444.350000,1650,Manual,19.55814


In [63]:
drivetrain = pd.get_dummies(data['Drivetrain'])
drivetrain.head()

,2-Wheel Drive,"2-Wheel Drive, Front",4-Wheel Drive,4-Wheel or All-Wheel Drive,All-Wheel Drive,Front-Wheel Drive,Part-time 4-Wheel Drive,Rear-Wheel Drive
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0


In [64]:
data = pd.concat([data, drivetrain], axis=1)
data

,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,...,TransType,Avg_MPG,2-Wheel Drive,"2-Wheel Drive, Front",4-Wheel Drive,4-Wheel or All-Wheel Drive,All-Wheel Drive,Front-Wheel Drive,Part-time 4-Wheel Drive,Rear-Wheel Drive
0,1985-01-01,Alfa Romeo,Spider Veloce 2000,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,4.0,2.0,15.695714,...,Manual,19.55814,0,0,0,0,0,0,0,1
1,1994-01-01,Alfa Romeo,Spider,Two Seaters,Manual 5spd,Rear-Wheel Drive,Premium,4.0,2.0,14.982273,...,Manual,19.55814,0,0,0,0,0,0,0,1
2,1994-01-01,Alfa Romeo,164,Compact Cars,Auto 4spd,Front-Wheel Drive,Premium,6.0,3.0,20.600625,...,Automatic,19.55814,0,0,0,0,0,1,0,0
3,1994-01-01,Alfa Romeo,164,Compact Cars,Manual 5spd,Front-Wheel Drive,Premium,6.0,3.0,18.311667,...,Manual,19.55814,0,0,0,0,0,1,0,0
4,1995-01-01,Alfa Romeo,164,Compact Cars,Auto 4spd,Front-Wheel Drive,Premium,6.0,3.0,20.600625,...,Automatic,19.55814,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36953,1992-01-01,Consulier Industries Inc,Consulier GTP,Two Seaters,Manual 5spd,Rear-Wheel Drive,Regular,4.0,2.2,15.695714,...,Manual,21.00000,0,0,0,0,0,0,0,1
36954,1991-01-01,Goldacre,Goldacre Limited,Two Seaters,Manual 5spd,Rear-Wheel Drive,Premium,6.0,2.5,18.311667,...,Manual,18.00000,0,0,0,0,0,0,0,1
36955,1992-01-01,Isis Imports Ltd,Morgan Plus 8,Two Seaters,Manual 5spd,Rear-Wheel Drive,Premium,8.0,3.9,16.480500,...,Manual,20.00000,0,0,0,0,0,0,0,1
36956,1992-01-01,PAS Inc - GMC,Pas-Syclone,Small Pickup Trucks,Auto 4spd,4-Wheel or All-Wheel Drive,Premium,6.0,4.3,21.974000,...,Automatic,15.00000,0,0,0,1,0,0,0,0


In [65]:
lexus = data[data['Make']=='Lexus']
audi = data[data['Make']=='Audi']
lexus

,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,...,TransType,Avg_MPG,2-Wheel Drive,"2-Wheel Drive, Front",4-Wheel Drive,4-Wheel or All-Wheel Drive,All-Wheel Drive,Front-Wheel Drive,Part-time 4-Wheel Drive,Rear-Wheel Drive
20638,1993-01-01,Lexus,LS 400,Midsize Cars,Auto 4spd,Rear-Wheel Drive,Premium,8.0,4.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20639,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20640,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Manual 5spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Manual,20.982368,0,0,0,0,0,0,0,1
20641,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,8.0,4.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20642,1994-01-01,Lexus,ES 300,Compact Cars,Auto 4spd,Front-Wheel Drive,Regular,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21030,1993-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
21031,1993-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Manual 5spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Manual,20.982368,0,0,0,0,0,0,0,1
21032,1993-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,8.0,4.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
21033,1993-01-01,Lexus,ES 300,Compact Cars,Auto 4spd,Front-Wheel Drive,Regular,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,1,0,0


In [66]:
lexus_audi = pd.concat([lexus, audi], axis=0)
lexus_audi

,Year,Make,Model,Vehicle_Class,Transmission,Drivetrain,Fuel_Type,Cylinders,Engine_Displacement,Fuel_Barrels_Year,...,TransType,Avg_MPG,2-Wheel Drive,"2-Wheel Drive, Front",4-Wheel Drive,4-Wheel or All-Wheel Drive,All-Wheel Drive,Front-Wheel Drive,Part-time 4-Wheel Drive,Rear-Wheel Drive
20638,1993-01-01,Lexus,LS 400,Midsize Cars,Auto 4spd,Rear-Wheel Drive,Premium,8.0,4.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20639,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20640,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Manual 5spd,Rear-Wheel Drive,Premium,6.0,3.0,18.311667,...,Manual,20.982368,0,0,0,0,0,0,0,1
20641,1994-01-01,Lexus,SC 300/SC 400,Subcompact Cars,Auto 4spd,Rear-Wheel Drive,Premium,8.0,4.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,0,0,1
20642,1994-01-01,Lexus,ES 300,Compact Cars,Auto 4spd,Front-Wheel Drive,Regular,6.0,3.0,18.311667,...,Automatic,20.982368,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8076,1993-01-01,Audi,S4,Compact Cars,Manual 5spd,4-Wheel or All-Wheel Drive,Premium,5.0,2.2,18.311667,...,Manual,20.293275,0,0,0,1,0,0,0,0
8077,1993-01-01,Audi,100 quattro,Compact Cars,Auto 4spd,4-Wheel or All-Wheel Drive,Premium,6.0,2.8,18.311667,...,Automatic,20.293275,0,0,0,1,0,0,0,0
8078,1993-01-01,Audi,100 quattro,Compact Cars,Manual 5spd,4-Wheel or All-Wheel Drive,Premium,6.0,2.8,18.311667,...,Manual,20.293275,0,0,0,1,0,0,0,0
8079,1993-01-01,Audi,90,Compact Cars,Auto 4spd,Front-Wheel Drive,Premium,6.0,2.8,19.388824,...,Automatic,20.293275,0,0,0,0,0,1,0,0


In [67]:
melted = pd.melt(data, id_vars=['Year','Make','Model'],
                 value_vars=['City_MPG','Highway_MPG','Combined_MPG'])
melted

,Year,Make,Model,variable,value
0,1985-01-01,Alfa Romeo,Spider Veloce 2000,City_MPG,19
1,1994-01-01,Alfa Romeo,Spider,City_MPG,19
2,1994-01-01,Alfa Romeo,164,City_MPG,14
3,1994-01-01,Alfa Romeo,164,City_MPG,15
4,1995-01-01,Alfa Romeo,164,City_MPG,14
...,...,...,...,...,...
110869,1992-01-01,Consulier Industries Inc,Consulier GTP,Combined_MPG,21
110870,1991-01-01,Goldacre,Goldacre Limited,Combined_MPG,18
110871,1992-01-01,Isis Imports Ltd,Morgan Plus 8,Combined_MPG,20
110872,1992-01-01,PAS Inc - GMC,Pas-Syclone,Combined_MPG,15


In [72]:
df = pd.DataFrame({'A': [[1, 2, 3], 'foo', [], [3, 4]], 'B': [1,'hola',50,'ADIU']})
df

,A,B
0,"[1, 2, 3]",1
1,foo,hola
2,[],50
3,"[3, 4]",ADIU


In [73]:
df.explode('A')

,A,B
0,1,1
0,2,1
0,3,1
1,foo,hola
2,NaN,50
3,3,ADIU
3,4,ADIU
